# Interactive Visualization of Daytimes and Workingtimes in Europe

## Data

In [2]:
import pandas as pd

RELOAD_DATA = False
eu_data_path = 'datasets/saved/eu_gpd.csv'
city_data_path = 'datasets/saved/city_data.csv'

### EU countries to capitals and GeoPandas data

In [3]:
from src.geo_utils import load_eu_countries_as_geopandas, get_eu_capitals, get_eu_city_data

if RELOAD_DATA:
    print('Reloading data from datasets/saved folder ...')
    eu_gpd = pd.read_csv(eu_data_path)
    top_city_data = pd.read_csv(city_data_path)
else:
    print('Generating data ...')
    eu_gpd = load_eu_countries_as_geopandas()
    eu_gpd.to_csv(eu_data_path, index=False)

    top_city_data = get_eu_city_data(3)
    top_city_data.to_csv(city_data_path, index=False)

eu_capitals = get_eu_capitals()
eu_iso_to_geo = eu_gpd.loc[:, ['iso_a3', 'geometry']]

Generating data ...


In [4]:
eu_gpd.head()

,pop_est,continent,name,iso_a3,gdp_md_est,geometry
21,5347896.0,Europe,Norway,NOR,403336,"MULTIPOLYGON (((15.14282 79.67431, 15.52255 80..."
43,67059887.0,Europe,France,FRA,2715518,"MULTIPOLYGON (((-51.65780 4.15623, -52.24934 3..."
110,10285453.0,Europe,Sweden,SWE,530883,"POLYGON ((11.02737 58.85615, 11.46827 59.43239..."
111,9466856.0,Europe,Belarus,BLR,63080,"POLYGON ((28.17671 56.16913, 29.22951 55.91834..."
112,44385155.0,Europe,Ukraine,UKR,153781,"POLYGON ((32.15944 52.06125, 32.41206 52.28869..."


In [5]:
top_city_data.head()

,population,CODE,country_ISO_A2,NAME,longitude,latitude,mercantor_x,mercantor_y,social_timezone,utc_sun_timezone_offset,longitudinal_diff,longitudinal_diff_km
0,1205492,BE001C,BE,Bruxelles/Brussel,4.351697,50.846557,4.844287e+05,6.594196e+06,Europe/Brussels,1.0,10.648303,851.864240
1,523591,BE002C,BE,Antwerpen,4.399708,51.221110,4.897733e+05,6.660499e+06,Europe/Brussels,1.0,10.600292,848.023352
2,383710,BE005C,BE,Liège,5.573611,50.645094,6.204516e+05,6.558754e+06,Europe/Brussels,1.0,9.426389,754.111104
3,1238438,BG001C,BG,Sofia,23.321736,42.697703,2.596164e+06,5.266072e+06,Europe/Sofia,2.0,6.678264,534.261128
4,345213,BG002C,BG,Plovdiv,24.749930,42.141854,2.755150e+06,5.182252e+06,Europe/Sofia,2.0,5.250070,420.005624


### Generate top n cities per country with timezone features

### Sunrise/Sunset (UTC) for the year 2022

In [6]:
from src.sun_data import get_sunset_sunrise_data
sun_data_df = get_sunset_sunrise_data(year=2022, country_capital=eu_capitals)

Could not process day 2022-06-16 00:00:00


In [7]:
import geopandas as gpd
sun_data_gpd = gpd.GeoDataFrame(sun_data_df.merge(eu_iso_to_geo, on='iso_a3', how='left').set_index('iso_a3'))

## Interactive visualization with Panel/Bokeh

In [82]:
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, ColumnDataSource, LabelSet
from bokeh.tile_providers import get_provider, Vendors
from bokeh.palettes import Plasma256
from bokeh.palettes import brewer
import json
import panel as pn
from datetime import datetime

#data_field = 'sunrise_UTC'
bokeh_tools = 'wheel_zoom, pan, hover, box_zoom, reset'

def get_bokeh_geodata_source(gpd_df):
    json_data = json.dumps(json.loads(gpd_df.to_json()))
    return GeoJSONDataSource(geojson = json_data)

def bokeh_plot_map(data):
    tooltips = [
        ('Country', '@country_ISO_A2'),
        ('City', '@NAME'),
        ('Relative position (dist) to timezone border', '@longitudinal_diff_km')
    ]
    p = figure(toolbar_location='right', tools=bokeh_tools, tooltips=tooltips, active_scroll ="wheel_zoom",
               title="Distance to eastern timezone meridian for large EU cities")
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None

    # ADD GEO STUFF FOR COUNTRIES AS A WHOLE -------------------------------------------------
    # geo_data_source = get_bokeh_geodata_source(data)
    #
    # values = data[data_field]
    # palette = brewer['OrRd'][8]
    # palette = palette[::-1]
    # #Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
    # color_mapper = LinearColorMapper(palette = palette, low = values.min(), high = values.max())
    # color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=500, height=20,
    #                      location=(0,0), orientation='horizontal')
    #  # p.patches('xs','ys', source=geo_data_source,
    # #           fill_color={'field': data_field, 'transform': color_mapper},
    #           line_color='blue',
    #           line_width=0.5,
    #           fill_alpha=0.8)
    #     p.add_layout(color_bar, 'below')



    # AD MAP TILES ---------------------------------------------------------------------------
    world_tile = get_provider(Vendors.STAMEN_TONER_BACKGROUND)

    # ADD TIMEZONE INFO PER TOP N CITIES FOR EACH COUNTRY ------------------------------------

    city_data_source = ColumnDataSource(data=top_city_data)
    city_color_mapper = LinearColorMapper(palette=Plasma256,
                                          low=top_city_data['longitudinal_diff_km'].min(),
                                          high=top_city_data['longitudinal_diff_km'].max())
    city_color_bar = ColorBar(color_mapper=city_color_mapper, label_standoff=8, width=500, height=20,
                     location=(0,0), orientation='horizontal')
    p.circle(x='mercantor_x', y='mercantor_y', source=city_data_source,
             color={'field': 'longitudinal_diff_km', 'transform': city_color_mapper}, size=10, fill_alpha=1)
    labels = LabelSet(x='mercantor_x', y='mercantor_y', x_offset=5, y_offset=5, text='NAME', source=city_data_source, text_color='cornflowerblue')
    p.add_layout(labels)
    p.add_layout(city_color_bar, 'below')
    p.add_tile(world_tile)
    return p

In [83]:
def map_visualization():
    map_pane = pn.pane.Bokeh(width=1000, height=1000)
    default_date = datetime.strptime('2022-01-01', '%Y-%m-%d').date()
    selected_date = pn.widgets.DatePicker(name='Date Picker', value=default_date)
    def update_map(event):
        d = selected_date.value
        selected_sundata = sun_data_gpd.query(f'day == {d.day} & month == {d.month} & year == {d.year}')
        map_pane.object = bokeh_plot_map(selected_sundata)
    selected_date.param.watch(update_map, 'value')
    selected_date.param.trigger('value')
    return pn.Column(selected_date, map_pane)

app = map_visualization()
app.show()

Launching server at http://localhost:5532
